In [10]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools
from IPython.display import Image, clear_output

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
import tensorflow as tf
import pandas as pd
from data_info import *
from preprocessing_helpers import *
from datetime import datetime
import math
%matplotlib inline
pd.set_option("display.precision", 2)
tf.random.set_seed(42)

In [2]:
all_data = extract_data(train_file, CSV_COLUMNS, interpolate=False)

In [23]:
all_data.describe().head()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,...,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,iq,sj
count,1456.00,1456.00,1262.00,1404.00,1434.00,1434.00,1443.00,1446.00,1446.00,1446.00,...,1446.00,1446.00,1413.00,1413.00,1436.00,1442.00,1434.00,1456.00,1456.00,1456.00
mean,2001.03,26.50,0.14,0.13,0.20,0.20,45.76,298.70,299.23,295.25,...,16.75,4.90,27.19,8.06,32.45,22.10,39.33,24.68,0.36,0.64
std,5.41,15.02,0.14,0.12,0.07,0.08,43.72,1.36,1.26,1.53,...,1.54,3.55,1.29,2.13,1.96,1.57,47.46,43.60,0.48,0.48
min,1990.00,1.00,-0.41,-0.46,-0.02,-0.06,0.00,294.64,294.89,289.64,...,11.72,1.36,21.40,4.53,26.70,14.70,0.00,0.00,0.00,0.00
25%,1997.00,13.75,0.04,0.05,0.16,0.14,9.80,297.66,298.26,294.12,...,15.56,2.33,26.30,6.51,31.10,21.10,8.70,5.00,0.00,0.00


In [27]:
sj_data = all_data[all_data['sj'] == 1]
total_absolute_error = {}
for attr_name in ['ndvi_ne', 'ndvi_nw', 'ndvi_se', 'ndvi_sw']:
    total_absolute_error[attr_name] = 0
    for index, row in sj_data.iterrows():
        if index > 0 and index < len(sj_data) -1 and not math.isnan(row[attr_name]):
            prev_row = sj_data.iloc[index - 1]
            next_row = sj_data.iloc[index + 1]
            if not math.isnan(next_row[attr_name]) and not math.isnan(prev_row[attr_name]):
                total_absolute_error[attr_name] += abs((next_row[attr_name] + prev_row[attr_name] )/2 - row[attr_name])
            
print(total_absolute_error['ndvi_ne']/sj_data.describe()['ndvi_ne']['count'])
print(total_absolute_error['ndvi_nw']/sj_data.describe()['ndvi_nw']['count'])
print(total_absolute_error['ndvi_se']/sj_data.describe()['ndvi_se']['count'])
print(total_absolute_error['ndvi_sw']/sj_data.describe()['ndvi_sw']['count'])

0.047045216838255036
0.05500962294419398
0.04941361314503814
0.04786313442202839


In [28]:
sj_data.describe()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,...,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,iq,sj
count,936.00,936.00,7.45e+02,887.00,917.00,917.00,927.00,930.00,930.00,930.00,...,930.00,930.00,930.00,930.00,930.00,930.00,930.00,936.00,936.0,936.0
mean,1998.83,26.50,5.79e-02,0.07,0.18,0.17,35.47,299.16,299.28,295.11,...,16.55,2.52,27.01,6.76,31.61,22.60,26.79,34.18,0.0,1.0
std,5.21,15.02,1.07e-01,0.09,0.06,0.06,44.61,1.24,1.22,1.57,...,1.56,0.50,1.42,0.84,1.72,1.51,29.33,51.38,0.0,0.0
min,1990.00,1.00,-4.06e-01,-0.46,-0.02,-0.06,0.00,295.94,296.11,289.64,...,11.72,1.36,22.84,4.53,26.70,17.80,0.00,0.00,0.0,1.0
25%,1994.00,13.75,4.50e-03,0.02,0.14,0.13,0.00,298.19,298.30,293.85,...,15.24,2.16,25.84,6.20,30.60,21.70,6.82,9.00,0.0,1.0
50%,1999.00,26.50,5.77e-02,0.07,0.18,0.17,20.80,299.25,299.38,295.46,...,16.85,2.46,27.23,6.76,31.70,22.80,17.75,19.00,0.0,1.0
75%,2003.00,39.25,1.11e-01,0.12,0.21,0.20,52.18,300.13,300.23,296.42,...,17.86,2.80,28.19,7.29,32.80,23.90,35.45,37.00,0.0,1.0
max,2008.00,53.00,4.93e-01,0.44,0.39,0.38,390.60,302.20,302.16,297.80,...,19.44,4.43,30.07,9.91,35.60,25.60,305.90,461.00,0.0,1.0


In [46]:
iq_data = all_data[all_data['iq'] == 1]
total_absolute_error = {}
for attr_name in ['ndvi_ne', 'ndvi_nw', 'ndvi_se', 'ndvi_sw']:
    total_absolute_error[attr_name] = 0
    for index, row in iq_data.iterrows():
        if index > 0 and index < len(iq_data) + len(sj_data) -1 and not math.isnan(row[attr_name]):
            prev_row = iq_data.iloc[index - len(sj_data) - 1]
            next_row = iq_data.iloc[index - len(sj_data) + 1]
            if not math.isnan(next_row[attr_name]) and not math.isnan(prev_row[attr_name]):
                total_absolute_error[attr_name] += abs((next_row[attr_name] + prev_row[attr_name] )/2 - row[attr_name])
            
print(total_absolute_error['ndvi_ne']/iq_data.describe()['ndvi_ne']['count'])
print(total_absolute_error['ndvi_nw']/iq_data.describe()['ndvi_nw']['count'])
print(total_absolute_error['ndvi_se']/iq_data.describe()['ndvi_se']['count'])
print(total_absolute_error['ndvi_sw']/iq_data.describe()['ndvi_sw']['count'])

0.07045492083172136
0.06984907232108319
0.06397035707930367
0.07540096841392657


In [47]:
iq_data.describe()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,...,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,iq,sj
count,520.00,520.00,517.00,517.00,517.00,517.00,516.00,516.00,516.00,516.00,...,516.00,516.00,483.00,483.00,506.00,512.00,504.00,520.00,520.0,520.0
mean,2005.00,26.50,0.26,0.24,0.25,0.27,64.25,297.87,299.13,295.49,...,17.10,9.21,27.53,10.57,34.00,21.20,62.47,7.57,1.0,0.0
std,2.92,15.03,0.08,0.08,0.08,0.09,35.22,1.17,1.33,1.42,...,1.45,2.45,0.92,1.54,1.33,1.26,63.25,10.77,0.0,0.0
min,2000.00,1.00,0.06,0.04,0.03,0.06,0.00,294.64,294.89,290.09,...,12.11,3.71,21.40,5.20,30.10,14.70,0.00,0.00,1.0,0.0
25%,2002.75,13.75,0.20,0.18,0.19,0.20,39.11,297.09,298.22,294.59,...,16.10,7.37,27.00,9.50,33.20,20.60,17.20,1.00,1.0,0.0
50%,2005.00,26.50,0.26,0.23,0.25,0.26,60.47,297.82,299.12,295.85,...,17.43,8.96,27.60,10.62,34.00,21.30,45.30,5.00,1.0,0.0
75%,2007.25,39.25,0.32,0.29,0.30,0.33,85.76,298.65,300.12,296.55,...,18.18,11.01,28.10,11.66,34.90,22.00,85.95,9.00,1.0,0.0
max,2010.00,53.00,0.51,0.45,0.54,0.55,210.83,301.64,302.93,298.45,...,20.46,16.03,30.80,15.80,42.20,24.20,543.30,116.00,1.0,0.0
